# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
city_data_df = pd.read_csv("city_data.csv")
city_data_df

In [ ]:
# Drop the Unnamed:0 column that was created when the .csv file was read into the notebook
city_data_df.drop(["Unnamed: 0"], axis=1, inplace=True)
city_data_df

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [ ]:
# Store latitude and longitude in locations
locations = city_data_df[["Lat", "Lng"]]

# Fill Nan values and convert to float
humidity = city_data_df["Humidity"].astype(float)

In [ ]:
# Customize the size of the figure
figure_layout = {
    "width": "600px",
    "height": "500px",
    "border": "1px solid black",
    "padding": "1px",
    "margin": "0 auto 0 auto"
}

In [ ]:
# Plot heatmap
fig = gmaps.figure(layout = figure_layout)

# Create heat layer
heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                    dissipating=True, max_intensity = 10,
                                    point_radius = 5)

# Add heat layer
fig.add_layer(heatmap_layer)

# Display figure
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
ideal_weather_df = city_data_df[(city_data_df["Max Temp"] >= 75) & (city_data_df["Max Temp"] <= 85)]
ideal_weather_df

### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Create dataframe and add empty column
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

In [ ]:
# Find the first hotel for each city located within 5,000 meters of the coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    #"location": "40.4018, 17.6335",
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

In [ ]:
# Loop through dataframe and add nearby hotel result

# Create a list of coordinates by using iterrows to iterate through hotel_df
for index, row in hotel_df.iterrows():
    
    # Get Lat/Lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response["results"]
    
    try:
        # 'name' is the key Google uses when providing the hotel name in the json
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, "Hotel Name"] = results[0]["Hotel Name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

In [ ]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(hotel_locations)

# Add the layer to the map
fig.add_layer(markers)
fig